# Notebook details

In [ ]:
def setup_notebook(fix_python_path=True, reduce_margins=True, plot_inline=True):
    if reduce_margins:
        # Reduce side margins of the notebook
        from IPython.core.display import display, HTML
        display(HTML("<style>.container { width:100% !important; }</style>"))

    if fix_python_path:
        # add egosocial to the python path
        import os, sys
        sys.path.extend([os.path.dirname(os.path.abspath('.'))])

    if plot_inline:
        # Plots inside cells
        %matplotlib inline
    
    global __file__
    __file__ = 'Notebook'

setup_notebook()

# Imports and Constants Definition

In [ ]:
# !/usr/bin/env python
# -*- coding: utf-8 -*-

import argparse
import json
import logging
import os

import IPython
import ipywidgets as widgets

import cv2

import egosocial
from egosocial import config
from egosocial.core.types import FaceClustering
from egosocial.utils.filesystem import check_directory 
from egosocial.utils.filesystem import create_directory
from egosocial.utils.filesystem import list_files_in_segment
from egosocial.utils.filesystem import list_segments
from egosocial.utils.logging import setup_logging
from egosocial.utils.parser import FACE_DETECTION
from egosocial.utils.parser import load_faces_from_file

DOMAINS = ['Attachent', 'Reciprocity', 'Mating', 'Heirarchical Power', 'Coalitional Group']
RELATIONS = [
    ['father-child', 'mother-child', 'grandpa-grandchild', 'grandma-grandchild'],
    ['friends', 'siblings', 'classmates'],
    ['lovers/spouses'],
    ['presenter-audience', 'teacher-student', 'trainer-trainee', 'leader-subordinate', 'customer-staff'],
    ['band members', 'dance team members', 'sport team members', 'colleages'],
]

In [ ]:
def main(*fake_args):
    entry_msg = 'Create social relation labels for egosocial photo-streams.'
    parser = argparse.ArgumentParser(description=entry_msg)

    parser.add_argument('--images_dir', required=True,
                        help='Directory containing social segments.')    
    parser.add_argument('--groups_dir', required=True,
                        help='Directory containing the groups information.')
    parser.add_argument('--face_detection_dir', required=True,
                        help='Directory containing face detection for social '
                             'segments.')
    parser.add_argument('--labels_dir', required=True,
                        help='Directory where to store the output labels')
    
    parser.add_argument('--groups_file_name', default='grouped_faces.json',
                        help='File name containing the groups information.')
    parser.add_argument('--labels_file_name', default='labels.json',
                        help='File name containing the labels information.') 
    parser.add_argument('--input_dir_extra', default='',
                        help="""Extra folders inside input segment directories.
                                Example: given a heirarchy 10/bar/foo/, 
                                call with --input_dir_extra bar,foo""")
    
    if not os.path.isdir(egosocial.config.TMP_DIR):
        os.mkdir(egosocial.config.TMP_DIR)

    setup_logging(egosocial.config.LOGGING_CONFIG,
                  log_dir=egosocial.config.LOGS_DIR)
    
    # TODO: implement correctly
    args = parser.parse_args(*fake_args)
    
    return args

# Main class

In [ ]:
def tagging_widget():
    
    grouped_relations = []
    all_relations = []
    
    unknown = 'others'
    
    for d_idx, domain in enumerate(DOMAINS):
        relation_bboxes = []
        
        for relation in RELATIONS[d_idx] + [unknown]:
            if relation != unknown:
                description = "{}".format(relation)            
            else:
                description = "{} {}".format(domain, unknown)
            
            bbox = widgets.Checkbox(
                description=description,
                value=False, 
                disabled=False,
            )
            
            relation_bboxes.append(bbox)
            all_relations.append(bbox)
        
        domain_relations = widgets.VBox(
            [widgets.Label(domain), widgets.VBox(relation_bboxes)]
        )            
        
        grouped_relations.append(domain_relations)
    
    result_widget = widgets.HBox(
        [widgets.VBox(grouped_relations[:3]), 
         widgets.VBox(grouped_relations[3:])]
    )
    
    return result_widget, all_relations

class SocialRelationsTagger:
    
    def __init__(
        self, 
        images_dir=None,
        groups_dir=None,
        groups_file_name=None,
        face_detection_dir=None,        
        labels_dir=None,
        labels_file_name=None,
        ):
        # TODO: add docstring
        self._images_dir = images_dir
        self._groups_dir = groups_dir
        self._groups_file_name = groups_file_name
        self._face_detection_dir = face_detection_dir
        self._labels_dir = labels_dir
        self._labels_file_name = labels_file_name
        self._segm_images_dir = None
        self._segm_labels_dir = None      
        
        self._face_detection_method = FACE_DETECTION.DOCKER_FACE
        self._pattern = FACE_DETECTION.get_file_pattern(self._face_detection_method)        

        self._current_segment_id = None
        self._current_groups = None
        self._current_frame_idx = None
        self._current_group_id = None
        self._current_faces = None
        self._segments = None
        self._segments_progress = None
        # default without folders inside segment
        self.segment_input_extra = []        
                
        # set up logging
        self._log = logging.getLogger(self.__class__.__name__)

    def process_all(self):
        # TODO: add docstring
        self._segments = sorted(list_segments(self._groups_dir), key=int)
        self._segments_progress = 0
        
        if self._segments:
            segment_id = self._segments[self._segments_progress]            
            self.process_segment(segment_id)
        
    def process_segment(self, segment_id):
        segment_id = str(segment_id)
        # TODO: add docstring
        self._log.info('Labeling samples for segment {}'.format(segment_id))

        # deal with inner folders inside segments
        image_dir_unrolled = [self._images_dir, segment_id]
        image_dir_unrolled.extend(self.segment_input_extra)
        # check input directory
        self._segm_images_dir = os.path.join(*image_dir_unrolled)
        check_directory(self._segm_images_dir, 'Input Segment')

        # create labels directory if necessary (also create segment
        # directory the first time)
        self._segm_labels_dir = os.path.join(self._labels_dir, segment_id)            
        create_directory(self._segm_labels_dir, 'Labels')
        
        self._current_segment_id = segment_id        
        self._current_groups = self._load_face_clustering(segment_id).groups
        self._current_faces = self._load_faces(segment_id)
        self._current_group_id = 0
        self._current_frame_idx = 0
        self._tagging_results = []
        self._widgets = None
        self._extra_widgets = None
        
        self._init_gui()
        self.refresh_ui()
    
    def _init_gui(self):
        # TODO: add docstring
        self._widgets = {}

        relations_w, relations_bbox = tagging_widget()
        self._widgets['relations'] = relations_w
        self._widgets['relations_bbox'] = relations_bbox
        
        if (self._current_group_id == len(self._current_groups) or 
            len(self._current_groups[self._current_group_id]) == 0):
            self._widgets['image'] = widgets.Label("No faces found!")
        else:
            image, ext = self._get_framed_face()
            self._widgets['image'] = widgets.Image(value=image, format=ext, width=480, height=640)
        
        # buttons
        next_frame_btn = widgets.Button(
            description='Next Frame',
            disabled=False,
            tooltip='Show next frame in the segment.',
            icon='forward'
        )
        prev_frame_btn = widgets.Button(
            description='Previous Frame',
            disabled=False,
            tooltip='Show previous frame in the segment.',
            icon='backward'
        )        
        submit_btn = widgets.Button(
            description='Submit',
            disabled=False,
            tooltip='Save tagging.',
            icon='check'
        )        
        next_segment_btn = widgets.Button(
            description='Next Segment',
            disabled=False,
            tooltip='Process next segment.',
            icon='forward',
        )
        prev_segment_btn = widgets.Button(
            description='Previous Segment',
            disabled=False,
            tooltip='Process previous segment.',
            icon='backward',
        )
        
        # frames forward/backward
        image_list_w = widgets.VBox(
            [widgets.Label("Segment {}".format(self._current_segment_id)),
             self._widgets['image'], 
             widgets.HBox([prev_frame_btn, next_frame_btn])]
        )
        
        self._widgets['faces_progress_bar'] = widgets.IntProgress(
            value=self._current_group_id,
            min=0, max=len(self._current_groups), 
            step=1, description='#Faces',
            orientation='horizontal'
        )
        
        # tagging
        tagging_subwidgets = [relations_w, widgets.HBox([submit_btn, self._widgets['faces_progress_bar']])]
        
        if self._segments:
            self._widgets['segment_progress_bar'] = widgets.IntProgress(
                value=self._segments_progress,
                min=0, max=len(self._segments), 
                step=1, description='#Segments',
                orientation='horizontal'
            )
            tagging_subwidgets.append(widgets.HBox([prev_segment_btn, next_segment_btn, self._widgets['segment_progress_bar']]))
        
        tagging_w = widgets.VBox(tagging_subwidgets)

        # complete tool
        self._widgets['tool'] = widgets.HBox(
            [image_list_w, tagging_w]
        )        
        
        # button callbacks
        def on_submit(b):
            if self._current_group_id == len(self._current_groups):
                return
            
            tagging_result = [
                bbox.description for bbox in self._widgets['relations_bbox'] if bbox.value
            ]
            
            group = self._current_groups[self._current_group_id][0]
            self._tagging_results.append(dict(group_id=group.group_id, labels=tagging_result))

            self._current_group_id += 1                
            if self._current_group_id < len(self._current_groups):
                # _init_gui marks progress automatically
                self._current_frame_idx = 0
                self._init_gui()
            else:
                # all groups are done, now we can save the results to disk
                self._store_labels()
                # mark progress manually
                self._widgets['faces_progress_bar'].value += 1

            self.refresh_ui()   

        def on_prev_frame(b):
            if self._current_group_id == len(self._current_groups):
                return
            
            new_frame_idx = max(self._current_frame_idx-1, 0)
            if new_frame_idx != self._current_frame_idx:
                self._current_frame_idx = new_frame_idx
                
                image, ext = self._get_framed_face()                
                self._widgets['image'].value = image
                self.refresh_ui()
                
        def on_next_frame(b):
            if self._current_group_id == len(self._current_groups):
                return
            
            group = self._current_groups[self._current_group_id]            
            
            if self._current_frame_idx < len(group)-1:
                self._current_frame_idx += 1
                
                image, ext = self._get_framed_face()
                self._widgets['image'].value = image                
                self.refresh_ui()
        
        def on_next_segment(b):            
            if self._segments and self._segments_progress < len(self._segments):
                self._segments_progress += 1
                if self._segments_progress < len(self._segments):                   
                    segment_id = self._segments[self._segments_progress]                    
                    self.process_segment(segment_id)
            
        def on_prev_segment(b):
            if self._segments and self._segments_progress > 0:
                self._segments_progress -= 1
                segment_id = self._segments[self._segments_progress]                    
                self.process_segment(segment_id)
            
        # connect callbacks
        submit_btn.on_click(on_submit)                
        next_frame_btn.on_click(on_next_frame)
        prev_frame_btn.on_click(on_prev_frame) 
        next_segment_btn.on_click(on_next_segment)
        prev_segment_btn.on_click(on_prev_segment)        
        
    def refresh_ui(self):
        IPython.display.clear_output(wait=True)
        display(self._widgets['tool'])  
        
    def _get_framed_face(self):        
        group = self._current_groups[self._current_group_id]        
        iface = group[self._current_frame_idx]
        
        image_name, ext = os.path.splitext(iface.image_name)
        src_image_name, face_id = image_name.rsplit('_', 1)
        src_image_name = src_image_name + ext
        face_id = int(face_id)        
        
        src_image_path = os.path.join(self._segm_images_dir, src_image_name)

        image = self._load_image(src_image_path)
        face = self._current_faces[src_image_name][face_id]
        left_top, bottom_right = (face.bbox.left, face.bbox.top), (face.bbox.left + face.bbox.width, face.bbox.top + face.bbox.height)
        color, thickness = (0, 255, 0), 2
        
        cv2.rectangle(image, left_top, bottom_right, color, thickness)
        
        _, image_array = cv2.imencode(ext, image)
        raw_image = image_array.tobytes()
        ext = ext[1:]
        
        return raw_image, ext
    
    def _load_face_clustering(self, segment_id):
        # check groups directory
        segm_groups_dir = os.path.join(self._groups_dir, segment_id)
        check_directory(segm_groups_dir, 'Groups Segment')
        
        groups_path = os.path.join(segm_groups_dir, self._groups_file_name)
        # TODO: add docstring
        self._log.debug('Loading groups %s from ' % groups_path)

        with open(groups_path) as json_file:
            groups_asjson = json.load(json_file)

        return FaceClustering.from_json(groups_asjson)

    def _load_image(self, image_path):
        # TODO: add docstring
        # read whole image
        image = cv2.imread(image_path)        
        return image
    
    def _load_faces(self, segment_id):        
        faces_by_image = {}
        self._log.debug('Faces in segment %s' % segment_id)
        
        for detection_path in list_files_in_segment(self._face_detection_dir,
                                                    segment_id,
                                                    file_pattern=self._pattern):
            self._log.debug('Loading faces from %s' % detection_path)
            faces = load_faces_from_file(detection_path,
                                         format=self._face_detection_method)
            # skip loop if no faces found
            if faces:
                image_name = faces[0].image_name
                faces_by_image[image_name] = faces

        return faces_by_image
    
    def _store_labels(self):        
        output_path = os.path.join(self._segm_labels_dir, self._labels_file_name)
        # TODO: add docstring
        self._log.debug('Saving labels to %s' % output_path)

        with open(output_path, 'w') as json_file:
            json.dump(self._tagging_results, json_file, indent=4)

In [ ]:
split = 'extended'

args = [
    "--images_dir", '/media/emasa/OS/Users/Emanuel/Downloads/NO_SYNC/Social Segments/images/' + split,
    "--groups_dir", '/media/emasa/OS/Users/Emanuel/Downloads/NO_SYNC/Social Segments/clustering_output_sync/' + split,
    "--face_detection_dir", '/media/emasa/OS/Users/Emanuel/Downloads/NO_SYNC/Social Segments/facedocker_detection/' + split,
    "--labels_dir", '/media/emasa/OS/Users/Emanuel/Downloads/NO_SYNC/Social Segments/labels/' + split,
#    "--input_dir_extra", "data",
]

conf = main(args)

tagger = SocialRelationsTagger(
    images_dir=conf.images_dir,
    groups_dir=conf.groups_dir,
    groups_file_name=conf.groups_file_name,
    face_detection_dir=conf.face_detection_dir,        
    labels_dir=conf.labels_dir,
    labels_file_name=conf.labels_file_name,
)

extra = conf.input_dir_extra
# work around for test folders, run with '--input_dir_extra data
# configure extra folders inside input segments (default empty)
tagger.segment_input_extra = extra.strip().split(',') if extra else []

all_segments_iter = tagger.process_all()